# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
#         print(root)
        # searches based on wild cards, there can be multiple directories, files  with same name but unique root
        files = glob.glob(os.path.join(root,'*.json'))
#         print(files)
#         print('abs path',os.path.abspath(files[0]))
        for f in files :
        #abs path from current working directory
            all_files.append(os.path.abspath(f))
    
    return all_files

In [4]:
os.listdir('data/song_data')

['A', '.ipynb_checkpoints']

In [5]:
os.listdir()

['create_tables.py',
 '.workspace-config.json',
 'etl.py',
 'README.md',
 'data',
 '__pycache__',
 '.ipynb_checkpoints',
 'test.ipynb',
 'data.zip',
 'etl.ipynb',
 'sql_queries.py']

In [6]:
for root, dirs, files in os.walk('data/song_data'):
    print(root)

data/song_data
data/song_data/A
data/song_data/A/A
data/song_data/A/A/A
data/song_data/A/A/A/.ipynb_checkpoints
data/song_data/A/A/B
data/song_data/A/A/.ipynb_checkpoints
data/song_data/A/A/C
data/song_data/A/B
data/song_data/A/B/A
data/song_data/A/B/B
data/song_data/A/B/.ipynb_checkpoints
data/song_data/A/B/C
data/song_data/A/.ipynb_checkpoints
data/song_data/.ipynb_checkpoints


In [7]:
# get_files('data/song_data')

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [8]:
song_files = get_files('data/song_data')

In [9]:
len(song_files)

72

In [10]:
song_files[0] #note the abs path and wildcard search through glob,join and  root by walk instead of for loop
#glob returns strings from directory we are searching so need of abs path

'/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json'

In [11]:
filepath = song_files[0]

In [12]:
df = pd.read_json(filepath, lines=True)

df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [13]:
song_data = df[['song_id','title','artist_id','year','duration']]

print(song_data)
song_data=song_data.values
song_data.tolist()

              song_id             title           artist_id  year   duration
0  SOMZWCG12A8C13C480  I Didn't Mean To  ARD7TVE1187B99BFB1     0  218.93179


[['SOMZWCG12A8C13C480',
  "I Didn't Mean To",
  'ARD7TVE1187B99BFB1',
  0,
  218.93179]]

### observe diffference in index and values selection

In [14]:
song_data = df[['song_id','title','artist_id','year','duration']]
song_data.set_index("song_id", inplace = True)

print(song_data)
song_data=song_data.values
song_data.tolist()

                               title           artist_id  year   duration
song_id                                                                  
SOMZWCG12A8C13C480  I Didn't Mean To  ARD7TVE1187B99BFB1     0  218.93179


[["I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]]

In [15]:
song_data = df[['song_id','title','artist_id','year','duration']]
print(song_data)
song_data=song_data.values
song_data=song_data.tolist()[0]
song_data

              song_id             title           artist_id  year   duration
0  SOMZWCG12A8C13C480  I Didn't Mean To  ARD7TVE1187B99BFB1     0  218.93179


['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [16]:
from sql_queries import *
song_table_insert

'\n    INSERT into songs(song_id,title,artist_id,year,duration) \n    VALUES (%s,%s,%s,%s,%s)\n    ON CONFLICT(song_id)\n    DO NOTHING;\n'

In [17]:
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

In [18]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [19]:
df

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


In [20]:
artist_data = df[[ 'artist_id', 'artist_name', 'artist_location','artist_latitude', 'artist_longitude']]
artist_data=artist_data.values[0].tolist()
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [21]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [22]:
log_files = get_files('data/log_data')


In [23]:
filepath = log_files[0]
filepath


'/home/workspace/data/log_data/2018/11/2018-11-30-events.json'

In [24]:
df = pd.read_json(filepath,lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [25]:
df['page'].unique()

array(['NextSong', 'Home', 'Logout', 'Settings', 'Downgrade', 'Login',
       'Help', 'Error', 'Upgrade'], dtype=object)

In [26]:
df = df[df['page']=='NextSong']
df['page'].unique()

array(['NextSong'], dtype=object)

In [27]:
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91


In [28]:
df.info() #int object

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330 entries, 0 to 387
Data columns (total 18 columns):
artist           330 non-null object
auth             330 non-null object
firstName        330 non-null object
gender           330 non-null object
itemInSession    330 non-null int64
lastName         330 non-null object
length           330 non-null float64
level            330 non-null object
location         330 non-null object
method           330 non-null object
page             330 non-null object
registration     330 non-null float64
sessionId        330 non-null int64
song             330 non-null object
status           330 non-null int64
ts               330 non-null int64
userAgent        330 non-null object
userId           330 non-null object
dtypes: float64(2), int64(4), object(12)
memory usage: 49.0+ KB


In [29]:
df['ts'][0]

1543537327796

In [30]:
pd.to_datetime(df['ts'],unit='ms')
#ime stamp is always a string representing the unix timestamp in milliseconds 
#milliseconds is counted from 1960 etc some sort of default clock in every programming language 
#thats how its  able to locate month etc 


0     2018-11-30 00:22:07.796
1     2018-11-30 01:08:41.796
2     2018-11-30 01:12:48.796
3     2018-11-30 01:17:05.796
4     2018-11-30 01:20:56.796
5     2018-11-30 01:24:32.796
6     2018-11-30 01:28:19.796
7     2018-11-30 01:32:00.796
9     2018-11-30 01:38:17.796
10    2018-11-30 01:39:53.796
11    2018-11-30 01:44:10.796
12    2018-11-30 01:48:36.796
13    2018-11-30 01:50:28.796
14    2018-11-30 01:55:35.796
15    2018-11-30 02:01:08.796
16    2018-11-30 02:05:24.796
17    2018-11-30 02:10:21.796
18    2018-11-30 02:15:54.796
19    2018-11-30 02:19:19.796
23    2018-11-30 03:06:30.796
24    2018-11-30 03:10:30.796
25    2018-11-30 03:13:03.796
26    2018-11-30 03:15:41.796
27    2018-11-30 03:21:09.796
30    2018-11-30 03:29:23.796
31    2018-11-30 03:32:46.796
33    2018-11-30 03:37:24.796
35    2018-11-30 03:39:31.796
36    2018-11-30 03:44:28.796
38    2018-11-30 03:49:50.796
                ...          
356   2018-11-30 17:49:45.796
357   2018-11-30 17:53:44.796
358   2018

In [31]:
t = pd.to_datetime(df['ts'],unit='ms')
t.head()

0   2018-11-30 00:22:07.796
1   2018-11-30 01:08:41.796
2   2018-11-30 01:12:48.796
3   2018-11-30 01:17:05.796
4   2018-11-30 01:20:56.796
Name: ts, dtype: datetime64[ns]

In [32]:
# Python program to demonstrate creating 
# pandas Datadaframe from lists using zip. 
    
import pandas as pd 
    
# List1 
Name = ['tom', 'krish', 'nick', 'juli'] 
    
# List2 
Age = [25, 30, 26, 22] 
    
# get the list of tuples from two lists. 
# and merge them by using zip(). 
list_of_tuples = list(zip(Name, Age)) 
    
# Assign data to tuples. 
list_of_tuples  
  
  
# Converting lists of tuples into 
# pandas Dataframe. 
df2 = pd.DataFrame(list_of_tuples,
                  columns = ['Name', 'Age']) 
     
# Print data. 
df2 

,Name,Age
0,tom,25
1,krish,30
2,nick,26
3,juli,22


In [33]:
#time_data={'hour','day','week','month','year','weekday'}
columns=['hour','day','week','month','year','weekday']


In [34]:
#dictionary to pandas dataframe easy way instead of tuples list zip 
#for reference another way of writing when we have multiple lists
time_data={} #dictionary
time_data['start_time']=list(t)
time_data['hour']=t.dt.hour.values.tolist()
time_data['day']=t.dt.day.values.tolist()
time_data['week']=t.dt.week.values.tolist()
time_data['month']=t.dt.month.values.tolist()
time_data['year']=t.dt.year.values.tolist()
time_data['weekday']=t.dt.weekday.values.tolist()
tdf=pd.DataFrame(time_data)
#tdf=tdf.astype({"start_time":datetime})
tdf.head(2)


,start_time,hour,day,week,month,year,weekday
0,2018-11-30 00:22:07.796,0,30,48,11,2018,4
1,2018-11-30 01:08:41.796,1,30,48,11,2018,4


In [35]:
# time_data = ()
# column_labels = ()

In [36]:
time_df = tdf
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-30 00:22:07.796,0,30,48,11,2018,4
1,2018-11-30 01:08:41.796,1,30,48,11,2018,4
2,2018-11-30 01:12:48.796,1,30,48,11,2018,4
3,2018-11-30 01:17:05.796,1,30,48,11,2018,4
4,2018-11-30 01:20:56.796,1,30,48,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [37]:
for i, row in time_df.iterrows():
    if i<2:
        print(row)
        print('list is',list(row))

start_time    2018-11-30 00:22:07.796000
hour                                   0
day                                   30
week                                  48
month                                 11
year                                2018
weekday                                4
Name: 0, dtype: object
list is [Timestamp('2018-11-30 00:22:07.796000'), 0, 30, 48, 11, 2018, 4]
start_time    2018-11-30 01:08:41.796000
hour                                   1
day                                   30
week                                  48
month                                 11
year                                2018
weekday                                4
Name: 1, dtype: object
list is [Timestamp('2018-11-30 01:08:41.796000'), 1, 30, 48, 11, 2018, 4]


In [38]:
for i, row in time_df.iterrows():
    #iterrows is generator which yields rows and columns
    #row is  pandas series iterator of each row in dataframe converted to list and inserted by loop
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [39]:
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91


In [40]:
df['level'].unique()

array(['free', 'paid'], dtype=object)

In [41]:
user_df = df[['userId','firstName','lastName','gender','level']]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [42]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [43]:
#song_data = df[['song_id','title','artist_id','year','duration']]
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

In [44]:
#artist_data = df[[ 'artist_id', 'artist_name', 'artist_location','artist_latitude', 'artist_longitude']]
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

In [45]:
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91


In [46]:
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91


In [47]:
for index,row in df.iterrows():
    print(index,row)
    break

0 artist                                               Stephen Lynch
auth                                                     Logged In
firstName                                                   Jayden
gender                                                           M
itemInSession                                                    0
lastName                                                      Bell
length                                                     182.857
level                                                         free
location                           Dallas-Fort Worth-Arlington, TX
method                                                         PUT
page                                                      NextSong
registration                                           1.54099e+12
sessionId                                                      829
song                                             Jim Henson's Dead
status                                                      

In [48]:
song_select

'\n    SELECT song_id,artists.artist_id \n    FROM songs \n    JOIN artists ON artists.artist_id=songs.artist_id\n    WHERE songs.title=%s \n    AND artists.artist_name=%s \n    AND songs.duration=%s \n'

In [49]:
# the user log may have songs that the songs table (only 1 song loaded) does not have any id or artists which returns none
# a user  may listen same songs
#multiple song table files may introduce index error because it repeates

In [50]:
df['ts']=pd.to_datetime(df['ts'],unit='ms')


In [51]:
for index, row in df.iterrows():
    #duration is matching with song length
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
#     print(index)
    #print(results)
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record into logdata
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId,\
                     row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [52]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.